# TSFEDL Models

In [1]:
import json
import os
import sys
sys.path.append('../')  ### to detect libraries in the parent directory
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from libraries.utils import *

import multiprocess
from multiprocess import Pool, cpu_count
from itertools import repeat
import time


pool = Pool(processes=(cpu_count() - 2))
print(cpu_count())


8


## Load data

In [2]:
# ############ configuration - trace ################
# ############################################


CODE = 'theft_protection'       ### application (code)       ###  'theft_protection', 'mamba2', 'lora_ducy'
BEHAVIOUR_FAULTY = 'faulty_data'            ### normal, faulty_data
BEHAVIOUR_NORMAL = 'normal'            ### normal, faulty_data
THREAD = 'single'           ### single, multi
VER = 3                     ### format of data collection

base_dir = '../../trace_data' ### can be replaced with 'csv', 'exe_plot', 'histogram'
normalbase_path = base_dir+f'/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR_NORMAL}'
faultybase_path = base_dir+f'/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR_FAULTY}'

print(normalbase_path)
print(faultybase_path)


################# configuration - diag ################
IS_VAR_WINDOW = False             ### True: varibale window size, False: fixed window size; wether to use variable window size or not

#####################################################


ref_samples_basepath = os.path.join(normalbase_path, 'diag_refsamples')
ref_var_samples_basepath = os.path.join(normalbase_path, 'diag_var_refsamples')
diag_subseq_basepath = os.path.join(faultybase_path, 'diag_subseq')
subseq_label_basepath = os.path.join(diag_subseq_basepath, 'subseq_labels')


print('ref_samples_path:\n', ref_samples_basepath)
print('ref_var_samples_path:\n', ref_var_samples_basepath)
print('diag_subseq_path:\n', diag_subseq_basepath)

######### get paths #######################
ref_samples_path = [os.path.join(ref_samples_basepath, x) for x in os.listdir(ref_samples_basepath)]
ref_var_samples_path = [os.path.join(ref_var_samples_basepath, x) for x in os.listdir(ref_var_samples_basepath)]   

train_varlist_path = os.listdir(normalbase_path)
train_varlist_path = [os.path.join(normalbase_path, x) for x in train_varlist_path if 'varlist' in x]

######### get paths #######################
paths_log, paths_traces, varlist_path, paths_label = get_paths(faultybase_path)

test_subseq_path = [os.path.join(diag_subseq_basepath, x) for x in os.listdir(diag_subseq_basepath)]
test_labels_path = [os.path.join(subseq_label_basepath, x) for x in os.listdir(subseq_label_basepath)]

# ### remove.Ds_store from all lists
train_varlist_path = [x for x in train_varlist_path if '.DS_Store' not in x]
varlist_path = [x for x in varlist_path if '.DS_Store' not in x]
paths_label = [x for x in paths_label if '.DS_Store' not in x]
ref_samples_path = [x for x in ref_samples_path if '.DS_Store' not in x]
ref_var_samples_path = [x for x in ref_var_samples_path if '.DS_Store' not in x]
test_subseq_path = [x for x in test_subseq_path if '.DS_Store' not in x if '.json' in x]
test_labels_path = [x for x in test_labels_path if '.DS_Store' not in x]


varlist_path.sort()

# print(paths_log)
# print(paths_traces)
# print(varlist_path)
# print(paths_label)

if IS_VAR_WINDOW:
    train_data_path = ref_var_samples_path
else:
    train_data_path = ref_samples_path

test_data_path = test_subseq_path

print('train_data:\n', train_data_path)
print(len(train_data_path))
print('test_data:\n', test_data_path)
print(len(test_data_path))
print('test_labels:\n', test_labels_path)



../../trace_data/theft_protection/single_thread/version_3/normal
../../trace_data/theft_protection/single_thread/version_3/faulty_data
ref_samples_path:
 ../../trace_data/theft_protection/single_thread/version_3/normal/diag_refsamples
ref_var_samples_path:
 ../../trace_data/theft_protection/single_thread/version_3/normal/diag_var_refsamples
diag_subseq_path:
 ../../trace_data/theft_protection/single_thread/version_3/faulty_data/diag_subseq
train_data:
 ['../../trace_data/theft_protection/single_thread/version_3/normal/diag_refsamples/379.json', '../../trace_data/theft_protection/single_thread/version_3/normal/diag_refsamples/396.json', '../../trace_data/theft_protection/single_thread/version_3/normal/diag_refsamples/115.json', '../../trace_data/theft_protection/single_thread/version_3/normal/diag_refsamples/400.json', '../../trace_data/theft_protection/single_thread/version_3/normal/diag_refsamples/142.json', '../../trace_data/theft_protection/single_thread/version_3/normal/diag_refsam

In [3]:
############# check varlist is consistent ############
############# only for version 3 ######################

if VER == 3 or VER == 4:
    check_con, _ = is_consistent([train_varlist_path[0]]+ varlist_path) ### compare with train varlist

    if check_con != False:
        to_number = read_json(varlist_path[0])
        from_number = mapint2var(to_number)
    else:
        ### load normal varlist
        print('loading normal varlist')
        to_number = read_json(train_varlist_path[0])
        from_number = mapint2var(to_number)



varlist 1 is consistent with varlist 0
varlist 2 is consistent with varlist 0
varlist 3 is consistent with varlist 0


In [4]:
to_number = read_json(train_varlist_path[0])
from_number = mapint2var(to_number)

In [5]:
# #### key finder ####
# from_number[44]

In [6]:
############ Get variable list ######################
sorted_keys = list(from_number.keys())
sorted_keys.sort()
var_list = [from_number[key] for key in sorted_keys]   ### get the variable list
# print(var_list)

## Prepare Training Data

In [7]:
### load all the reference samples (fixed window size)
### the ref_samples: list of list of events, list of intervals for the subseq of size 500
ref_samples = []
for ref_sample_path in train_data_path:
    ref_trace = read_traces(ref_sample_path)
    ref_samples.append(ref_trace)
    # print(ref_trace)
    # break

In [8]:
np.array(ref_samples).shape

(438, 2, 500)

In [9]:
### make subseq of 50 events, with sliding interval of 1
WINDOW = 50

def prepare_data(ref_trace, WINDOW):
    ref_events = ref_trace[0]
    ref_intervals = ref_trace[1]
    # print(len(ref_events))
    # print(len(ref_intervals))
    ### we take one less event for training because we need last event as label

    X_train_event = []
    Y_train_event = []
    for i in range(len(ref_events)-WINDOW): 
        # print(i)
        # print(ref_events[i])
        # sub_seq_events.append(ref_events[i:i+WINDOW])
        # sub_seq_intervals.append(ref_intervals[i:i+WINDOW])

        ### both events and intervals are taken as input
        # _x_train = [ref_events[i:i+WINDOW], ref_intervals[i:i+WINDOW]]
        # _x_train = np.array(_x_train)
        # _x_train = np.transpose(_x_train)
        # _y_train = np.array([ref_events[i+WINDOW], ref_intervals[i+WINDOW]])

        ### only events are taken as input
        _x_train_event = [ref_events[i:i+WINDOW]]
        _y_train_event = [ref_events[i+WINDOW]]
        # _y_train_event = [ref_events[i+WINDOW], ref_intervals[i+WINDOW]]

        X_train_event.append(_x_train_event)
        Y_train_event.append(_y_train_event)

    return [X_train_event, Y_train_event]


results = pool.starmap(prepare_data, zip(ref_samples[0:], repeat(WINDOW)))
# print(results)

# sub_seq_events = []
# sub_seq_intervals = []
X_train_event = []
Y_train_event = []
# X_train_interval = []
# Y_train_interval = []
for result in results:
    # print(len(result))
    # print(len(result[0]))
    # print(result[1])

    # for events_list in result[0]:
    #     X_train_event.append(events_list[0])

    # for events_list in result[1]:
    #     Y_train_event.append(events_list[0])

    X_train_event.extend(result[0])
    Y_train_event.extend(result[1])
    # break
        

In [ ]:
# ### make subseq of 50 events, with sliding interval of 1
# WINDOW = 50

# sub_seq_events = []
# sub_seq_intervals = []
# X_train_event = []
# Y_train_event = []
# X_train_interval = []
# Y_train_interval = []
# for ref_trace in ref_samples[:100]:
#     # print(ref_trace)
#     ref_events = ref_trace[0]
#     ref_intervals = ref_trace[1]
#     # print(len(ref_events))
#     # print(len(ref_intervals))
#     ### we take one less event for training because we need last event as label


#     for i in range(len(ref_events)-WINDOW): 
#         # print(i)
#         # print(ref_events[i])
#         # sub_seq_events.append(ref_events[i:i+WINDOW])
#         # sub_seq_intervals.append(ref_intervals[i:i+WINDOW])

#         ### both events and intervals are taken as input
#         # _x_train = [ref_events[i:i+WINDOW], ref_intervals[i:i+WINDOW]]
#         # _x_train = np.array(_x_train)
#         # _x_train = np.transpose(_x_train)
#         # _y_train = np.array([ref_events[i+WINDOW], ref_intervals[i+WINDOW]])

#         ### only events are taken as input
#         _x_train_event = [ref_events[i:i+WINDOW]]
#         _y_train_event = [ref_events[i+WINDOW]]
#         # _y_train_event = [ref_events[i+WINDOW], ref_intervals[i+WINDOW]]

#         X_train_event.append(_x_train_event)
#         Y_train_event.append(_y_train_event)


#         # ### only intervals are taken as input
#         # _x_train_interval = [ref_intervals[i:i+WINDOW]]
#         # _y_train_interval = [ref_intervals[i+WINDOW]]
#         # X_train_interval.append(_x_train_interval)
#         # Y_train_interval.append(_y_train_interval)
        


#     # break

In [10]:
print(np.array(X_train_event).shape)
# print(np.array(Y_train_event).shape)
# print(np.array(X_train_interval).shape)
# print(np.array(Y_train_interval).shape)

(197100, 1, 50)


In [11]:
### preprocess training data
X_train_event = np.array(X_train_event)
Y_train_event = np.array(Y_train_event)

X_train_event = X_train_event.reshape(X_train_event.shape[0], X_train_event.shape[2], X_train_event.shape[1])
Y_train_event = Y_train_event.reshape(Y_train_event.shape[0], Y_train_event.shape[1], 1)

### shuffle the data
X_train_event, Y_train_event = shuffle(X_train_event, Y_train_event, random_state=0)

### split the data in train, validation and test sets
X_train_event, x_test_event, Y_train_event, y_test_event = train_test_split(X_train_event, Y_train_event, test_size=0.2, random_state=0)
# X_val, X_test, Y_val, Y_test = train_test_split(X_val, Y_val, test_size=0.5, random_state=0)

print(X_train_event.shape)
print(Y_train_event.shape)

print(x_test_event.shape)
print(y_test_event.shape)

# ### preprocess training data
# X_train_interval = np.array(X_train_interval)
# Y_train_interval = np.array(Y_train_interval)

# X_train_interval = X_train_interval.reshape(X_train_interval.shape[0], X_train_interval.shape[2], X_train_interval.shape[1])
# Y_train_interval = Y_train_interval.reshape(Y_train_interval.shape[0], Y_train_interval.shape[1], 1)

# ### shuffle the data
# X_train_interval, Y_train_interval = shuffle(X_train_interval, Y_train_interval, random_state=0)

# ### split the data in train, validation and test sets
# X_train_interval, x_test_interval, Y_train_interval, y_test_interval = train_test_split(X_train_interval, Y_train_interval, test_size=0.2, random_state=0)

# print(X_train_interval.shape)
# print(Y_train_interval.shape)

# print(x_test_interval.shape)
# print(y_test_interval.shape)




(157680, 50, 1)
(157680, 1, 1)
(39420, 50, 1)
(39420, 1, 1)


In [12]:
X_train_event.reshape(X_train_event.shape[0], -1).reshape(X_train_event.shape).shape

(157680, 50, 1)

In [13]:
### normalize data
from sklearn.preprocessing import MinMaxScaler
import joblib

### check if scaler exists
if os.path.exists(f"./scalers/minmaxscaler_{CODE}.gz"):
    scaler = joblib.load(f"./scalers/minmaxscaler_{CODE}.gz")
    print('scaler loaded')

    print(X_train_event[0])
    X_train_event = scaler.transform(X_train_event.reshape(X_train_event.shape[0], -1)).reshape(X_train_event.shape)
    x_test_event = scaler.transform(x_test_event.reshape(x_test_event.shape[0], -1)).reshape(x_test_event.shape)
    print(X_train_event[0])
else:
    scaler = MinMaxScaler()

    print(X_train_event[0])
    X_train_event = scaler.fit_transform(X_train_event.reshape(X_train_event.shape[0], -1)).reshape(X_train_event.shape)
    x_test_event = scaler.transform(x_test_event.reshape(x_test_event.shape[0], -1)).reshape(x_test_event.shape)
    print(X_train_event[0])

    # print(X_train_event.shape)
    # print(x_test_event.shape)

    scaler_filename = f"./scalers/minmaxscaler_{CODE}.gz"
    joblib.dump(scaler, scaler_filename) 
    print('scaler saved')


scaler loaded
[[ 7]
 [ 8]
 [ 9]
 [ 6]
 [ 7]
 [ 8]
 [ 9]
 [10]
 [11]
 [12]
 [ 6]
 [ 7]
 [ 8]
 [ 9]
 [13]
 [14]
 [15]
 [ 6]
 [ 7]
 [ 8]
 [ 9]
 [ 6]
 [ 7]
 [ 8]
 [ 9]
 [ 6]
 [ 7]
 [ 8]
 [ 9]
 [ 6]
 [ 7]
 [ 8]
 [ 9]
 [10]
 [11]
 [12]
 [ 6]
 [ 7]
 [ 8]
 [ 9]
 [13]
 [14]
 [15]
 [ 6]
 [ 7]
 [ 8]
 [ 9]
 [ 6]
 [ 7]
 [ 8]]
[[0.11111111]
 [0.22222222]
 [0.33333333]
 [0.        ]
 [0.11111111]
 [0.22222222]
 [0.33333333]
 [0.44444444]
 [0.55555556]
 [0.66666667]
 [0.        ]
 [0.11111111]
 [0.22222222]
 [0.33333333]
 [0.77777778]
 [0.88888889]
 [1.        ]
 [0.        ]
 [0.11111111]
 [0.22222222]
 [0.33333333]
 [0.        ]
 [0.11111111]
 [0.22222222]
 [0.33333333]
 [0.        ]
 [0.11111111]
 [0.22222222]
 [0.33333333]
 [0.        ]
 [0.11111111]
 [0.22222222]
 [0.33333333]
 [0.44444444]
 [0.55555556]
 [0.66666667]
 [0.        ]
 [0.11111111]
 [0.22222222]
 [0.33333333]
 [0.77777778]
 [0.88888889]
 [1.        ]
 [0.        ]
 [0.11111111]
 [0.22222222]
 [0.33333333]
 [0.        ]
 [0.11111111]

In [16]:
y_test_event[0]

array([[57]])

## Build and Train DL Models

In [14]:
import tensorflow as tf
import TSFEDL.models_keras as tsfedl

FINE_TUNE = True

if not FINE_TUNE:
    print('training new model')
    #### build model ####
    input = tf.keras.Input(shape=(50,1))
    model = tsfedl.HuangMeiLing(input_tensor=input, include_top=False)
    x = model.output
    x = tf.keras.layers.LSTM(units=20)(x)
    ### Add the top module
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(50)(x)
    # x = tf.keras.layers.Dense(10)(x)
    x = tf.keras.layers.Dense(1)(x)
    out = tf.keras.layers.Reshape([1, 1])(x)

    ### create new model
    forecaster_event = tf.keras.Model(inputs=input, outputs=out, name="forecaster")
    # forecaster_interval = tf.keras.Model(inputs=input, outputs=out, name="forecaster")

    # print(model.summary())
    print(forecaster_event.summary())

else:
    print('fine tuning model')
    ### load the model
    model = tf.keras.models.load_model('./trained_models/forecaster_events_minmax_mamba2.keras')
    model.summary()



fine tuning model


Model: "forecaster"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 50, 1)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ zero_padding1d (ZeroPadding1D)  │ (None, 52, 1)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 7, 48)          │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 6, 48)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ zero_padding1d_1                │ (None, 12, 48)         │             0 │
│ (ZeroPadding1D)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 3, 256)         │        86,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 2, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 20)             │        22,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 50)             │         1,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            51 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 1, 1)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 330,905 (1.26 MB)

 Trainable params: 110,301 (430.86 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 220,604 (861.74 KB)

In [15]:
if not FINE_TUNE:
    ### train
    forecaster_event.compile(loss='mae', optimizer='adam', metrics=['mae', 'mse'])

    # history = forecaster.fit(X_train, Y_train, epochs=100, batch_size=32, validation_split=0.2)
    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
            # tf.keras.callbacks.ModelCheckpoint(filepath='./trained_models/forecaster_event.{epoch:02d}-{loss:.2f}.h5', monitor="loss", save_best_only=True),
            tf.keras.callbacks.TensorBoard(log_dir='./logs_forecaster_event'),]
    history_event = forecaster_event.fit(X_train_event, Y_train_event, epochs=50, batch_size=64, validation_split=0.2, callbacks=callbacks)

    # ### train
    # forecaster_interval.compile(loss='mae', optimizer='adam', metrics=['mae', 'mse'])
    # callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
    #         # tf.keras.callbacks.ModelCheckpoint(filepath='./trained_models/forecaster_interval.{epoch:02d}-{loss:.2f}.h5', monitor="loss", save_best_only=True),
    #         tf.keras.callbacks.TensorBoard(log_dir='./logs_forecaster_interval'),]
    # history_interval = forecaster_interval.fit(X_train_interval, Y_train_interval, epochs=50, batch_size=64, validation_split=0.2, callbacks=callbacks)
else:
    model.compile(loss='mae', optimizer=tf.keras.optimizers.Adam(1e-04), metrics=['mae', 'mse'])

    # history = forecaster.fit(X_train, Y_train, epochs=100, batch_size=32, validation_split=0.2)
    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
            # tf.keras.callbacks.ModelCheckpoint(filepath='./trained_models/forecaster_event.{epoch:02d}-{loss:.2f}.h5', monitor="loss", save_best_only=True),
            tf.keras.callbacks.TensorBoard(log_dir='./logs'),]
    history_event = model.fit(X_train_event, Y_train_event , epochs=50, batch_size=128, validation_split=0.2, callbacks=callbacks)




Epoch 1/50
986/986 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 26.5549 - mae: 26.5549 - mse: 996.3474 - val_loss: 8.0262 - val_mae: 8.0262 - val_mse: 100.9496
Epoch 2/50
986/986 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 6.7025 - mae: 6.7025 - mse: 77.3401 - val_loss: 3.9362 - val_mae: 3.9362 - val_mse: 39.4559
Epoch 3/50
986/986 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 3.5639 - mae: 3.5639 - mse: 35.1384 - val_loss: 2.1984 - val_mae: 2.1984 - val_mse: 21.9473
Epoch 4/50
986/986 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 2.0378 - mae: 2.0378 - mse: 20.3578 - val_loss: 1.7708 - val_mae: 1.7708 - val_mse: 17.3741
Epoch 5/50
986/986 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 1.6261 - mae: 1.6261 - mse: 16.0178 - val_loss: 1.4613 - val_mae: 1.4613 - val_mse: 14.2306
Epoch 6/50
986/986 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 1.3935 - mae: 1.3935 - mse: 13.3541 - val_loss: 1.2970 - val_mae: 1.2970 - val_mse: 11.8328
Epoch 7/50
986/986 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 1.2156 - mae: 1.2156 - m

In [16]:
### test ###
if not FINE_TUNE:
    print('testing new model')
    # new_model = tf.keras.models.load_model('./trained_models/forecaster_events.keras')
    y_pred_event = forecaster_event.predict(x_test_event)
    # y_pred_interval = forecaster_interval.predict(x_test_interval)

    print(y_pred_event.shape)
    # print(y_test_interval.shape)
else:
    print('testing fine tuned model')
    y_pred_event = model.predict(x_test_event)
    # y_pred_interval = model.predict(x_test_interval)

    print(y_pred_event.shape)
    # print(y_test_interval.shape)

testing fine tuned model
1232/1232 ━━━━━━━━━━━━━━━━━━━━ 1s 675us/step
(39420, 1, 1)


In [17]:
# for i in range(10):
#     print(y_test_interval[i], y_pred_interval[i])

for i in range(10):
    print(y_test_event[i], y_pred_event[i])

[[7]] [[6.993089]]
[[7]] [[6.993089]]
[[8]] [[7.9932127]]
[[9]] [[8.9982195]]
[[6]] [[5.9869666]]
[[10]] [[9.998362]]
[[12]] [[12.0107765]]
[[8]] [[7.9932127]]
[[12]] [[12.00213]]
[[9]] [[8.9982195]]


In [18]:
### testing
correct = []
incorrect = []

if y_test_event.shape[1] == 2:
    for i in range(len(y_test_event)):
        yt_event = y_test_event[i][0]
        yp_event = y_pred_event[i][0]

        yt_interval = y_test_event[i][1]
        yp_interval = y_pred_event[i][1]

        # print(yt_event, yp_event)
        # print(yt_interval, yp_interval)
        yt_event = yt_event.reshape(1,)
        yp_event = yp_event.reshape(1,)
        yt_interval = yt_interval.reshape(1,)
        yp_interval = yp_interval.reshape(1,)

        if np.abs(yt_event-yp_event) < 1 and np.abs(yt_interval-yp_interval) < 20:
            # print('correct prediction')
            correct.append(y_test_event[i])
        else:
            # print('incorrect prediction')
            incorrect.append(y_test_event[i])
            # print(yt, yp)
        # break

else:
    for i in range(len(y_test_event)):

        yt_event = y_test_event[i]
        yp_event = y_pred_event[i]


        yt_event = yt_event.reshape(1,)
        yp_event = yp_event.reshape(1,)

        if np.abs(yt_event-yp_event) < 1:
            # print('correct prediction')
            correct.append(y_test_event[i])
        else:
            # print('incorrect prediction')
            incorrect.append(y_test_event[i])
            # print(yt, yp)

        
        # break

print('correct:', len(correct))
print('incorrect:', len(incorrect))

### accuracy
accuracy = len(correct)/len(y_test_event)
print('accuracy:', accuracy)
### mae
mae = np.mean(np.abs(y_test_event-y_pred_event))
print('mae:', mae)
### mse
mse = np.mean((y_test_event-y_pred_event)**2)
print('mse:', mse)

correct: 38564
incorrect: 856
accuracy: 0.978285134449518
mae: 0.08147209426099912
mse: 0.23548434887606223


In [19]:
y_test_event.shape

(39420, 1, 1)

## save model

In [20]:
if not FINE_TUNE:
    print('saving new model')

    ### check if existing model is there
    model_name = f'./trained_models/forecaster_events_minmax_{CODE}.keras'
    if os.path.exists(model_name):
        print('model exists')
        raise FileExistsError
    else:
        ### save the model
        forecaster_event.save(model_name)
        # forecaster_interval.save('./trained_models/forecaster_intervals_1.keras')
        # forecaster.save_weights('./trained_models/forecaster_events.weights.h5')
else:
    print('saving fine tuned model')
    model_name = f'./trained_models/forecaster_events_minmax_mamba+theft.keras'

    if os.path.exists(model_name):
        print('model exists')
        raise FileExistsError
    else:
        model.save(model_name)
    # model.save('./trained_models/forecaster_intervals_1.keras')
    # model.save_weights('./trained_models/forecaster_events.weights.h5')


saving fine tuned model
